In [1]:
import pandas as pd

# 1. WA sales
wa = pd.read_csv('wa_cannabis_sales - Sheet1.csv')
wa['Sales Last Month'] = (
    wa['Sales Last Month']
      .replace(r'[\$,]', '', regex=True)
      .astype(float)
)

wa['Period'] = pd.to_datetime(wa['Period'], format='mixed')

# 2–3. CO product manufacturers (retail & medical)
prod_r = pd.read_csv('Product Manufacturers - Retail.csv', parse_dates=['Date Updated'])
prod_m = pd.read_csv('Product Manufacturers - Medical.csv', parse_dates=['Date Updated'])
prod = pd.concat([prod_r, prod_m], ignore_index=True)

# 5–8. CO stores & cultivations (retail & medical)
stores_r = pd.read_csv('Colorado_Stores - Retail.csv', parse_dates=['Date Updated'])
stores_m = pd.read_csv('Colorado_Stores - Medical.csv', parse_dates=['Date Updated'])
cultiv_r = pd.read_csv('Colorado_Cultivations - Retail.csv', parse_dates=['Date Updated'])
cultiv_m = pd.read_csv('Colorado_Cultivations - Medical.csv', parse_dates=['Date Updated'])
stores   = pd.concat([stores_r, stores_m],   ignore_index=True)
cultiv   = pd.concat([cultiv_r, cultiv_m],   ignore_index=True)

# 4. price per ounce (wide → long)
ppd = pd.read_csv('price_per_ounce_mass_2025.csv')
ppd['CCCLastUpdated'] = pd.to_datetime(ppd['CCCLastUpdated'], format='%m/%d/%Y')
price_long = ppd.melt(
    id_vars=['CCCLastUpdated','SOLDDATE'],
    value_vars=['CONSUMER_OZ','PATIENT_OZ','GRANDTOTAL_OZ'],
    var_name='Category', value_name='Price_per_oz'
)
price_long['SOLDDATE'] = pd.to_datetime(price_long['SOLDDATE'], format='%m/%Y')

# 9. CO historical sales (clean header row)

import pandas as pd

# ——— 1. Read & skip junk ———
# skiprows=2 drops row 0 (title) and row 1 (“Intentionally left blank”)
# header=None so pandas doesn’t try to infer column names
# 1. Read raw (skip title + blank line)
state_raw = pd.read_csv(
    'Colorado Marijuana_Sales_2014_To_2024_Report.xlsx - State Report.csv',
    skiprows=2, header=None, dtype=str
)

# 2. Promote row 0 → header
new_header = state_raw.iloc[0].str.replace('\n', ' ').str.strip().tolist()
df = state_raw[1:].copy()
df.columns = new_header

# 3. Rename to clean names
df = df.rename(columns={
    'Month':      'Month',
    'Year':       'Year',
    'Total Medical  Marijuana Sales ¹': 'Medical_Sales',
    'Total Retail  Marijuana Sales ²':  'Retail_Sales',
    'Total  Marijuana Sales':           'Total_Sales'
})

# 4. Now coerce Month/Year → numeric, drop any bad rows
df = df[pd.to_numeric(df['Month'], errors='coerce').notnull() &
        pd.to_numeric(df['Year'],  errors='coerce').notnull()].copy()
df['Month'] = df['Month'].astype(int)
df['Year']  = df['Year'].astype(int)

# 5. Strip $/commas & convert to float
for col in ['Medical_Sales','Retail_Sales','Total_Sales']:
    df[col] = df[col].replace(r'[\$,]', '', regex=True).astype(float)

# 6. Build a proper Date
df['Date'] = pd.to_datetime({
    'year':  df['Year'],
    'month': df['Month'],
    'day':   1
})


# Now you can plot state['Date'] vs. state['Total_Sales'] without errors


# 10. AD retail delivery mass
ad = pd.read_csv('ad_retail_delivery_sales_mass_2025.csv', parse_dates=['SaleDate'])


In [3]:
state = df


In [4]:
# app.py
import dash
from dash import html, dcc, Input, Output
import plotly.express as px

app = dash.Dash(__name__)
# get a clean list of counties (drop None, NaN, empty‐strings)
counties = [
    c for c in wa['County'].unique()
    if isinstance(c, str) and c.strip()
]

# build your options
wa_options = [{'label': c, 'value': c} for c in sorted(counties)]

dcc.Dropdown(
    id='wa-county',
    options=wa_options,
    value=wa_options[0]['value'],  # pick a sensible default
    clearable=False
)

# Precompute figures
fig_sales = px.bar(state, x='Date', y='Total_Sales', title='Colorado Total Sales (2014–2024)')
fig_price = px.line(price_long, x='SOLDDATE', y='Price_per_oz', color='Category',
                    title='Average Price per oz over Time')

app.layout = html.Div([
    html.H1("Cannabis Market Dashboard"),
    html.Div([
        html.Div([
            dcc.Graph(figure=fig_sales)
        ], style={'width':'48%','display':'inline-block'}),
        html.Div([
            dcc.Graph(figure=fig_price)
        ], style={'width':'48%','display':'inline-block'})
    ]),
    html.H2("Washington State Sales Last Month"),
    dcc.Dropdown(id='wa-county',
                 options= wa_options,
                 value=wa['County'].unique()[0]),
    dcc.Graph(id='wa-sales-chart')
])

@app.callback(
    Output('wa-sales-chart','figure'),
    Input('wa-county','value')
)
def update_wa_chart(county):
    df = wa[wa['County']==county]
    fig = px.bar(df, x='Business Name', y='Sales Last Month',
                 title=f"Sales in {county} (Last Month)")
    return fig

if __name__=='__main__':
    app.run(debug=True,port=8051)
